In [ ]:
%matplotlib inline
import numpy as np
import skimage.io
import matplotlib.pyplot as plt
import skimage.morphology
import skimage.segmentation
import os
import utils.evaluation
import pandas as pd
import seaborn as sb

In [ ]:
base_dir = "/data1/image-segmentation/BBBC022/"

dir_gtruth = base_dir + "new_renamed_annotations/"

tag = "01"

dilation_parameter = 3

ref_out_dir_label = base_dir + "unet/experiments/" + tag + "/out/segm/"

out_dir_label = base_dir + "unet/experiments/" + tag + "/out/segm/" 

In [ ]:
def show(ground_truth, prediction, IOU, threshold=0.5, image_name="N"):
    
    # Create diff map
    diff = np.zeros(ground_truth.shape + (3,))
    A = ground_truth.copy()
    B = prediction.copy()
    A[A > 0] = 1
    B[B > 0] = 1
    D = A - B
    #diff[D > 0,:2] = 1
    #diff[D < 0,1:] = 1
    
    # Object-level errors
    C = IOU.copy()
    C[C>=threshold] = 1
    C[C<threshold] = 0
    missed = np.where(np.sum(C,axis=1) == 0)[0]
    extra = np.where(np.sum(C,axis=0) == 0)[0]

    for m in missed:
        diff[ground_truth == m+1, 0] = 1
    for e in extra:
        diff[prediction == e+1, 2] = 1
    
    # Display figures
    fig, ax = plt.subplots(1, 4, figsize=(18,6))
    ax[0].imshow(ground_truth)
    ax[0].set_title("True objects:"+str(len(np.unique(ground_truth))))
    ax[1].imshow(diff)
    ax[1].set_title("Segmentation errors:"+str(len(missed)))
    ax[2].imshow(prediction)
    ax[2].set_title("Predicted objects:"+str(len(np.unique(prediction))))
    ax[3].imshow(IOU)
    ax[3].set_title(image_name)

In [ ]:
def compute_results(ground_truth, prediction, results, image_name):
    
    # Compute Intersection over Union
    IOU = utils.evaluation.intersection_over_union(ground_truth, prediction)
    show(ground_truth, prediction, IOU, image_name=image_name)

    # Calculate precision at all thresholds
    for t in np.arange(0.5, 1.0, 0.05):
        p = utils.evaluation.precision_at(t, IOU)
        res = {"Image": image_name, "Threshold": t, "Precision": p}
        row = len(results)
        results.loc[row] = res
        
    return results

In [ ]:
def get_false_negatives(ground_truth, prediction, results, image_name, threshold=0.7):
    
    # Compute Intersection over Union
    IOU = utils.evaluation.intersection_over_union(ground_truth, prediction)

    true_objects = len(np.unique(ground_truth))
    if true_objects <= 1:
        return results
        
    area_true = np.histogram(ground_truth, bins=true_objects)[0][1:]
    true_objects -= 1
    
    # Identify False Negatives
    matches = IOU > threshold
    false_negatives = np.sum(matches, axis=1) == 0  # Missed objects

    data = np.asarray([ 
        area_true.copy(), 
        np.array(false_negatives, dtype=np.int32)
    ])

    results = pd.concat([results, pd.DataFrame(data=data.T, columns=["Area", "False_Negative"])])
        
    return results

In [ ]:
def get_splits_and_merges(ground_truth, prediction, results, image_name):
    
    IOU = utils.evaluation.intersection_over_union(ground_truth, prediction)
    matches = IOU > 0.3
    merges = np.sum(matches, axis=0) > 1
    splits = np.sum(matches, axis=1) > 1
    r = {"Image_Name":image_name, "Merges":np.sum(merges), "Splits":np.sum(splits)}
    results.loc[len(results)+1] = r
    return results

In [ ]:
all_images = os.listdir(ref_out_dir_label)

results = pd.DataFrame(columns=["Image", "Threshold", "Precision"])
false_negatives = pd.DataFrame(columns=["False_Negative", "Area"])
splits_merges = pd.DataFrame(columns=["Image_Name", "Merges","Splits"])

for image_name in all_images:
    # Load ground truth data
    ground_truth = skimage.io.imread(dir_gtruth + image_name)
    
    # Transform to label matrix
    ground_truth = skimage.morphology.label(ground_truth[:,:,0])
    
    # Load predictions
    prediction = skimage.io.imread(out_dir_label + image_name)
    prediction = skimage.morphology.dilation(prediction, skimage.morphology.square(dilation_parameter))
    
    # Compute evaluation metrics
    results = compute_results(ground_truth, prediction, results)
    false_negatives = get_false_negatives(ground_truth, prediction, results)
    splits_merges = get_splits_and_merges(ground_truth, prediction, results)

In [ ]:
# Display average precision results
average_precision = results.groupby("Threshold").mean().reset_index()
average_precision

In [ ]:
# Plot average precision results
sb.regplot(data=average_precision, x="Threshold", y="Precision", order=3, ci=None)

In [ ]:
# Compute and print Mean Average Precision
mean_average_precision = average_precision["Precision"].mean()
print("MAP:", mean_average_precision)

In [ ]:
# Summarize False Negatives by area
false_negatives = false_negatives[false_negatives["False_Negative"] == 1]

false_negatives.groupby(
    pd.cut(
        false_negatives["Area"], 
        [0,250,625,900,10000], # Area intervals
        labels=["Micronuclei","Small nuclei","Normal nuclei","Large nuclei"],
    )
)["False_Negative"].sum()

In [ ]:
# Summarize splits and merges
print("Splits:",np.sum(splits_merges["Splits"]))
print("Merges:",np.sum(splits_merges["Merges"]))